# Fine-tuning RadBert #
Full parameter fine tuning of RadBERT model (RadBERT-RoBERTa-4m model from the RadBERT paper). Fine tuning carried out on the multi-label muti-class classification of reports, where each report can have multiple labels (For ex, a report can havel label consolidation-right and consolidation-2)

In [1]:
import os
import time, datetime
import codecs
from itertools import product

import random
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

/home/users/pranav.rao/miniconda3/envs/TutorialCuda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Seeding everything for reproducibility
def seed_everything(seed: int):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 1000)

torch.set_printoptions(linewidth=200)

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
device='cuda:1'

## Forward pass ##
Implementing RadBERTMultiClassMulti Label PyTorch Model

In [6]:
class RadBERTMultiClassMultiLabel(nn.Module):
    """
    RadBERTMultiClassMultiLabel: Model expects batches of natural language sentences, will
    classify reports with multiple label
    """
    def __init__(self, num_classes, checkpoint, device):
        super().__init__()
        self.num_classes = num_classes
        self.checkpoint = checkpoint
        self.device = device

        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint)
        self.transformer_encoder = AutoModel.from_pretrained(self.checkpoint)
        self.transformer_encoder_hidden_size = self.transformer_encoder.config.hidden_size
        self.linear_classifier = nn.Linear(self.transformer_encoder_hidden_size, self.num_classes)
    
    def forward(self, x):
        tokenized_inp = self.tokenizer(x, padding=True, truncation=True, return_tensors='pt').to(self.device)
        encoder_out = self.transformer_encoder(**tokenized_inp)
        logits = self.linear_classifier(encoder_out.last_hidden_state[:, 0, :])
        return logits


In [7]:
checkpoint = 'UCSD-VA-health/RadBERT-RoBERTa-4m'
labels_subset = "normal tuberculosis opacity bronchialdilation density parenchymalopacity ett aorticenlargement mediastinalwidening mediastinalmass\
        copd prominentbronchovascularmarkings bronchitis markings vascularprominence interval interstitiallungdisease bluntedcp effusion cardiomegaly\
        consolidation subtle_normal peffusion lineandtube thickening haziness hilarprominence hilar inhomogenousopacity rotation\
        calcification unfoldedaorta bandlikeopacity aorticcalcification aorticknucklecalcification fibrosis suture cardiacshift degenspine nodule\
        pneumonia inspiration fracture pneumonitis justfibrosis lesion nonaorticcalcification tuberculosispure pleuralthickening feedingtube".split()
num_classes = len(labels_subset)

In [ ]:
radbert_multi_model = RadBERTMultiClassMultiLabel(num_classes, checkpoint, device).to(device)

In [ ]:
print(radbert_multi_model)
print(list(map(lambda x : x.shape, radbert_multi_model.parameters())))

## Custom Loss function ##
Custom loss function for multi class multi label classification to handle uncertain tags (tags have value 0 -> absent, 1 -> present, -100 -> uncertain)

In [ ]:
class MultiClassMultiLabel(nn.Module):
    def __init__(self, uncertain_label):
        super(MultiClassMultiLabel, self).__init__()
        self.uncertain_label = uncertain_label
    
    def forward(self, output, target):
        certain_mask = (target != self.uncertain_label)
        loss_func = nn.MultiLabelSoftMarginLoss(weight=certain_mask.type(torch.float))
        return loss_func(output, target)

In [ ]:
multiclass_multilabel_loss = MultiClassMultiLabel(-100).to(device)

### Testing MultiClassMultiLabel loss function ###

In [ ]:
logit_tensor = torch.Tensor([[-1.0, 2.0, 1.0, 5.0, -3.0], [4.0, -2.0, 1.0, -1.0, 2.5]]).to(device)
target_tensor_act = torch.Tensor([[0, 1, -100, 0, 0], [1, -100, 0, 0, 1]]).to(device)

In [ ]:
#certain_mask = (target_tensor_act != -100)
#print(certain_mask.type(torch.float))
#loss_func = nn.MultiLabelSoftMarginLoss(weight=(certain_mask).type(torch.float))
#print(loss_func(logit_tensor, target_tensor_act))
print(multiclass_multilabel_loss(logit_tensor, target_tensor_act))

In [ ]:
import math
from math import log, exp

def log_sigmoid(x):
    return log(1/(1+exp(-1*x)))

In [ ]:
loss1 = -1 * (1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.0) + 0.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(-5.0) + 1.0 * log_sigmoid(3.0)) / 5.0
loss2 = -1 * (1.0 * log_sigmoid(4.0) + 0.0 * log_sigmoid(2.0) + 1.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.5)) / 5.0

print(loss1)
print(loss2)
print((loss1 + loss2)/2.0)

## Custom DataLoader and Dataset ##
Read csv file, get the report from path and prepare the data

In [ ]:
class ReportTagsDataset(Dataset):
    def __init__(self, tags_csv_file, report_base_path, labels_subset=None, text_transform=None, target_transform=None):
        self.report_base_path = report_base_path
        self.tags_csv_file = tags_csv_file

        self.tags_df = pd.read_csv(self.tags_csv_file)
        self.column_names = list(self.tags_df.columns.values)
        self.column_names[0] = 'filename'
        self.tags_df.columns = self.column_names

        self.labels_subset = labels_subset
        self.text_transform = text_transform
        self.target_transform = target_transform
    
    def __len__(self):
        return self.tags_df.shape[0]
    
    def __getitem__(self, index):
        report_path = os.path.join(self.report_base_path, self.tags_df.iloc[index, 0].split('/')[-1] + '.txt')
        #report_text = open(report_path).read()
        report_text = codecs.open(report_path, 'r', encoding='utf-8', errors='ignore').read()
        if self.labels_subset is None:
            target_list = torch.Tensor(list(self.tags_df.iloc[index][1:]))
        else:
            target_list = torch.Tensor(list(self.tags_df[self.labels_subset].iloc[index]))
        return report_text, target_list

In [ ]:
#report_base_path = "/models_common_e2e/cxr_data/reports/training"
train_reports_base_path = '/home/users/pranav.rao/MiniTasks/Radbert/data/train'
test_reports_base_path = '/home/users/pranav.rao/MiniTasks/Radbert/data/test'
train_tags_file = '/home/users/pranav.rao/Downloads/report_tags_25k_train.csv'
test_tags_file = '/home/users/pranav.rao/Downloads/report_tags_25k_test.csv'

In [ ]:
train_data = ReportTagsDataset(train_tags_file, train_reports_base_path, labels_subset=labels_subset)
test_data = ReportTagsDataset(test_tags_file, test_reports_base_path, labels_subset=labels_subset)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)
#train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

### Testing Dataset and DataLoader ###

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))

In [ ]:
print(train_features)
print(train_labels)
print(f"Feature batch shape: {len(train_features)}")
print(f"Labels batch shape: {len(train_labels)}")
report_text = train_features[0]
print(report_text)
label = train_labels[0]
print(f"Label: {label}")

In [ ]:
with torch.no_grad():
    logit_tensor = radbert_multi_model(train_features)

In [ ]:
print(logit_tensor)
print(logit_tensor.shape)

## Fine Tuning ##
Fine tuning the RadBERT model for tags prediction task

### Adam Optimizer ###
Using Adam optimizer with learning rate 3e-5, beta1 = 0.9, beta2 = 0.99, l2 weight decay of 0.01

In [ ]:
lr = 3e-5
beta1 = 0.9
beta2 = 0.99
l2_weight_decay = 0.01

In [ ]:
optimizer = torch.optim.Adam(radbert_multi_model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=l2_weight_decay)

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = radbert_multi_model(inputs)

        # Compute the loss and its gradients
        loss = multiclass_multilabel_loss(outputs, labels)
        loss.backward()
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 50 == 49:
            last_loss = running_loss / 50 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

### The full fine-tuning loog ###

In [ ]:
total_epochs = 5

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))

for epoch in range(total_epochs):
    # Make sure gradient tracking is on, and do a pass over the data
    print('EPOCH {}:'.format(epoch + 1))
    radbert_multi_model.train(True)
    avg_loss = train_one_epoch(epoch, writer)
    model_path = '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_{}_{}'.format(timestamp, epoch)
    torch.save(radbert_multi_model.state_dict(), model_path)

    # Set the model to evaluation mode, disabling dropout and using population, statistics for batch normalization
    running_vloss = 0.0
    radbert_multi_model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(test_dataloader):
            vinputs, vlabels = vdata
            vlabels = vlabels.to(device)
            voutputs = radbert_multi_model(vinputs)
            vloss = multiclass_multilabel_loss(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch + 1)
    writer.flush()

## Analyzing fine-tuned models ##

In [8]:
sentence_list = ["The report shows small right-sided pleural effusion", "The report shows small left-sided pleural effusion",\
    "The report shows large right-sided pleural effusion", "The report shows large left-sided pleural effusion",\
    "There are no abnormalities in the report",\
    "There is severe consolidation in the left side","There is severe consolidation in the right side",\
    "There is mild consolidation in the right side", "There is mild consolidation in the left side"
]

sentence1_base = "A <SizeModifier> <AbnormalReport> can be seen in the report in the <LocationModifier> part"
sentence2_base = "The report shows a <SizeModifier> <LocationModifier> <AbnormalReport>"
size_modifiers = ['small', 'large']
loc_modifiers = ['upper-left', 'lower-left', 'right-sided', 'left-sided']
abnormal_report = ['pleural effusion']

l1 = [sentence1_base.replace('<SizeModifier>', size_mod).replace('<LocationModifier>', loc_mod).replace('<AbnormalReport>', ab_rep) for size_mod, loc_mod, ab_rep in product(size_modifiers, loc_modifiers, abnormal_report)]
l2 = [sentence2_base.replace('<SizeModifier>', size_mod).replace('<LocationModifier>', loc_mod).replace('<AbnormalReport>', ab_rep) for size_mod, loc_mod, ab_rep in product(size_modifiers, loc_modifiers, abnormal_report)]

negative_sentences = ['The report shows no pleural effusion', 'The report shows no consolidation on any side']
all_sentence_list = l1 + l2 + negative_sentences + sentence_list[4:]

In [9]:
print('\n'.join(all_sentence_list))

A small pleural effusion can be seen in the report in the upper-left part
A small pleural effusion can be seen in the report in the lower-left part
A small pleural effusion can be seen in the report in the right-sided part
A small pleural effusion can be seen in the report in the left-sided part
A large pleural effusion can be seen in the report in the upper-left part
A large pleural effusion can be seen in the report in the lower-left part
A large pleural effusion can be seen in the report in the right-sided part
A large pleural effusion can be seen in the report in the left-sided part
The report shows a small upper-left pleural effusion
The report shows a small lower-left pleural effusion
The report shows a small right-sided pleural effusion
The report shows a small left-sided pleural effusion
The report shows a large upper-left pleural effusion
The report shows a large lower-left pleural effusion
The report shows a large right-sided pleural effusion
The report shows a large left-sid

In [10]:
def get_sentence_embeddings(model, input):
    with torch.no_grad():
        tokenized_inp = model.tokenizer(input, padding=True, truncation=True, return_tensors='pt').to(device)
        encoder_out = model.transformer_encoder(**tokenized_inp)
    return encoder_out.last_hidden_state[:, 0, :]

In [11]:
def calc_cosine_sim_matrix(sentence_embeddings):
    cosine_sim_matrix = F.cosine_similarity(sentence_embeddings.unsqueeze(1), sentence_embeddings.unsqueeze(0), dim=2)
    return cosine_sim_matrix.cpu().detach().numpy()

In [12]:
model_path = '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20230914_160336_0'
radbert_multi_model = RadBERTMultiClassMultiLabel(num_classes, checkpoint, device).to(device)
radbert_multi_model.load_state_dict(torch.load(model_path))

Some weights of RobertaModel were not initialized from the model checkpoint at UCSD-VA-health/RadBERT-RoBERTa-4m and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [13]:
all_embeddings = get_sentence_embeddings(radbert_multi_model, all_sentence_list)

In [14]:
print(all_embeddings)
print(all_embeddings.shape)

tensor([[ 2.8910e-01, -5.2085e-01,  2.6253e-01,  ..., -1.8994e+00,  1.7182e-01,  1.3341e-02],
        [ 2.8928e-01, -5.4022e-01,  2.6898e-01,  ..., -1.9484e+00,  1.8694e-01,  2.6540e-03],
        [ 2.5743e-01, -5.3418e-01,  2.6586e-01,  ..., -1.8983e+00,  2.0071e-01, -5.6288e-02],
        ...,
        [ 4.0400e-01, -4.9373e-01, -2.8056e-01,  ..., -1.2606e+00, -4.3174e-03, -5.0392e-01],
        [ 4.3134e-01, -4.1540e-01, -2.8964e-01,  ..., -1.2568e+00, -1.1965e-03, -5.4829e-01],
        [ 4.4349e-01, -4.3328e-01, -2.7744e-01,  ..., -1.2366e+00,  4.1675e-02, -5.2168e-01]], device='cuda:1')
torch.Size([23, 768])


In [15]:
cosine_sim = calc_cosine_sim_matrix(all_embeddings)
pd.DataFrame(cosine_sim)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.000000,0.999362,0.998332,0.998653,0.997801,0.998091,0.995831,0.996154,0.983204,0.979679,0.979361,0.980066,0.982224,0.980238,0.978228,0.978952,0.835743,0.546055,0.377079,0.851285,0.846197,0.839327,0.845008
1,0.999362,1.000000,0.998027,0.998405,0.996324,0.997885,0.994678,0.995062,0.983346,0.981498,0.979830,0.980501,0.981407,0.981093,0.977705,0.978427,0.839366,0.548812,0.378454,0.848724,0.843807,0.838068,0.843557
2,0.998332,0.998027,1.000000,0.999480,0.996540,0.997042,0.997856,0.997323,0.983783,0.980915,0.983602,0.983302,0.983361,0.981940,0.982964,0.982605,0.837907,0.548808,0.380906,0.852061,0.848010,0.840563,0.845241
3,0.998653,0.998405,0.999480,1.000000,0.996919,0.997489,0.997374,0.997885,0.984088,0.981301,0.982959,0.983830,0.983676,0.982357,0.982280,0.983130,0.837925,0.546504,0.380839,0.850527,0.845046,0.837743,0.843712
4,0.997801,0.996324,0.996540,0.996919,1.000000,0.999346,0.998370,0.998721,0.981196,0.976589,0.977573,0.978383,0.984514,0.981260,0.980614,0.981406,0.828108,0.539589,0.376278,0.856504,0.850981,0.837227,0.843240
5,0.998091,0.997885,0.997042,0.997489,0.999346,1.000000,0.997928,0.998356,0.982041,0.979062,0.978660,0.979453,0.984260,0.982671,0.980580,0.981401,0.832623,0.542873,0.377468,0.854485,0.849156,0.836687,0.842494
6,0.995831,0.994678,0.997856,0.997374,0.998370,0.997928,1.000000,0.999473,0.981284,0.977302,0.981292,0.981076,0.985090,0.982373,0.984784,0.984471,0.829851,0.542189,0.379882,0.857028,0.852585,0.838364,0.843347
7,0.996154,0.995062,0.997323,0.997885,0.998721,0.998356,0.999473,1.000000,0.981551,0.977648,0.980607,0.981573,0.985344,0.982731,0.984031,0.984946,0.829844,0.539733,0.379700,0.855425,0.849545,0.835537,0.841809
8,0.983204,0.983346,0.983783,0.984088,0.981196,0.982041,0.981284,0.981551,1.000000,0.998734,0.997494,0.997905,0.997682,0.997897,0.994989,0.995401,0.881359,0.592676,0.418623,0.845569,0.840733,0.833136,0.838781
9,0.979679,0.981498,0.980915,0.981301,0.976589,0.979062,0.977302,0.977648,0.998734,1.000000,0.996861,0.997200,0.995149,0.997864,0.993045,0.993392,0.885705,0.597680,0.421948,0.841118,0.836497,0.830509,0.835895


## Analysing the predictions ##

In [16]:
def get_predictions(model, input):
    with torch.no_grad():
        tokenized_inp = model.tokenizer(input, padding=True, truncation=True, return_tensors='pt').to(device)
        encoder_out = model.transformer_encoder(**tokenized_inp)
        logits = model.linear_classifier(encoder_out.last_hidden_state[:, 0, :])
        return logits.cpu().detach()

In [17]:
def analyse_predictions(prediction_list, sentence_list):
    cols = ['Sentence', 'Best label', 'Best label score', 'Positive label']
    values = list()
    for (report, pred) in zip(sentence_list, prediction_list.numpy()):
        best_label = labels_subset[pred.argmax()]
        best_label_score = pred.max()
        positive_label = ','.join([labels_subset[i] for i, e in enumerate(pred) if e > 0.0])
        values.append([report, best_label, best_label_score, positive_label])
    return pd.DataFrame(values, columns=cols)

In [18]:
predictions = get_predictions(radbert_multi_model, all_sentence_list)

In [19]:
print(predictions)
print(predictions.shape)

tensor([[-2.0112,  1.7688,  0.9222,  ..., -1.5026, -0.9682, -1.3388],
        [-2.0314,  1.7700,  0.9226,  ..., -1.5127, -0.9895, -1.3558],
        [-1.9688,  1.7081,  0.8782,  ..., -1.4596, -0.9512, -1.2824],
        ...,
        [-2.6790,  1.9450,  1.5734,  ..., -1.4788, -1.4831, -1.8072],
        [-2.5836,  1.7072,  1.3661,  ..., -1.5444, -1.5293, -1.7637],
        [-2.6173,  1.7095,  1.4057,  ..., -1.5698, -1.5230, -1.7579]])
torch.Size([23, 50])


In [20]:
analyse_predictions(predictions, all_sentence_list)

,Sentence,Best label,Best label score,Positive label
0,A small pleural effusion can be seen in the report in the upper-left part,tuberculosis,1.768771,"tuberculosis,opacity,bronchialdilation,parenchymalopacity,effusion"
1,A small pleural effusion can be seen in the report in the lower-left part,tuberculosis,1.769983,"tuberculosis,opacity,parenchymalopacity,effusion"
2,A small pleural effusion can be seen in the report in the right-sided part,tuberculosis,1.708109,"tuberculosis,opacity,parenchymalopacity,effusion"
3,A small pleural effusion can be seen in the report in the left-sided part,tuberculosis,1.683801,"tuberculosis,opacity,parenchymalopacity,effusion"
4,A large pleural effusion can be seen in the report in the upper-left part,tuberculosis,1.807574,"tuberculosis,opacity,bronchialdilation,parenchymalopacity,effusion"
5,A large pleural effusion can be seen in the report in the lower-left part,tuberculosis,1.812663,"tuberculosis,opacity,parenchymalopacity,effusion"
6,A large pleural effusion can be seen in the report in the right-sided part,tuberculosis,1.749918,"tuberculosis,opacity,parenchymalopacity,effusion"
7,A large pleural effusion can be seen in the report in the left-sided part,tuberculosis,1.725523,"tuberculosis,opacity,parenchymalopacity,effusion"
8,The report shows a small upper-left pleural effusion,tuberculosis,1.423523,"tuberculosis,opacity,effusion"
9,The report shows a small lower-left pleural effusion,tuberculosis,1.397375,"tuberculosis,opacity,effusion"


## Bigger reports ##

In [21]:
dataset1_template = """xr- chest pa  view
findings
lungs: normal.
trachea: normal.
carina: normal.
right and left main bronchi: normal.
pleura: normal.
heart: normal.
right heart border: normal.
left heart border: normal.
pulmonary bay: normal.
pulmonary hila: normal.
aorta: normal.
thoracic spine: normal.
other visualized bones: normal.
visualized soft tissues: normal.
diaphragm: normal.
visualized abdomen:  normal.
visualized neck: normal."""

dataset1_pleural_issue="""
xr- chest pa view
findings
lungs: normal.
trachea: normal.
carina: normal.
right and left main bronchi: normal.
pleura: left costophrenic angle is blunted with thin stripe of homogenous opacity along left lateral chest wall.
heart: normal.
right heart border: normal.
left heart border: normal.
pulmonary bay: normal.
pulmonary hila: normal.
aorta: normal.
thoracic spine: normal.
other visualized bones: normal.
visualized soft tissues: normal.
diaphragm: normal.
visualized abdomen:  normal.
visualized neck: normal."""

dataset2_template="""6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:
results:
post cabg status.
no focal lesion seen in the lung parenchyma.
cp angles and domes of the diaphragm are normal.
both hila are normal. pulmonary vasculature is normal.
cardiac size and configuration is normal.
trachea is central; no mediastinal shift is seen.
bony thorax and soft tissues of the chest wall are normal.
impression: no abnormality detected in the view obtained.
"""

dataset3_template="""
x-ray chest (pa view)
the cardio thoracic ratio is normal.
the heart size and configuration are within normal limits.
the aortic arch is normal.
the lung fields show normal broncho-vascular markings.
both the pulmonary hila are normal in size.
the costophrenic and cardiophrenic recesses and the domes of
diaphragm are normal.
the bones and soft tissues of the chest wall show no abnormality.
impression : normal study.
dr.shakthi kumar
radiologist
ss
________________________________________________________
"""

In [22]:
all_embeddings2 = get_sentence_embeddings(radbert_multi_model, [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

In [23]:
print(all_embeddings2)
print(all_embeddings2.shape)

tensor([[ 0.4996,  0.5371, -0.7925,  ..., -1.3068,  0.1698, -0.7473],
        [ 0.7888, -0.7633,  0.0157,  ..., -2.0855,  0.2804, -1.2100],
        [ 0.3844,  0.1055, -0.9388,  ..., -1.5859,  0.3282, -1.0624],
        [ 0.4478,  0.2891, -0.8577,  ..., -1.7261,  0.2003, -1.0294]], device='cuda:1')
torch.Size([4, 768])


In [24]:
cosine_sim2 = calc_cosine_sim_matrix(all_embeddings2)
pd.DataFrame(cosine_sim2)

,0,1,2,3
0,1.000000,0.612040,0.969163,0.839009
1,0.612040,1.000000,0.612884,0.508108
2,0.969163,0.612884,1.000000,0.852399
3,0.839009,0.508108,0.852399,1.000000


In [25]:
predictions2 = get_predictions(radbert_multi_model, [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

In [26]:
print(predictions2)
print(predictions2.shape)

tensor([[ 2.1747, -4.1587, -3.8791, -4.1505, -3.3335, -4.2148, -3.4409, -2.8449, -3.6764, -3.8710, -4.1954, -4.1398, -4.2942, -4.3024, -4.2388, -3.4118, -4.1029, -4.0680, -4.3708, -3.6305, -4.1921,
         -3.3833, -4.2908, -3.7445, -4.1937, -4.2991, -4.1057, -4.4298, -4.4533, -3.3358, -4.2838, -3.8641, -4.5122, -4.4124, -4.3368, -4.5680, -3.9125, -4.1258, -3.7593, -4.5899, -4.8654, -4.1373,
         -4.0339, -4.6261, -4.3576, -4.6067, -4.4585, -4.9090, -4.2935, -4.3236],
        [-2.4192,  0.6862,  0.2971, -1.0279, -3.1733, -1.1821, -2.8068, -2.4515, -2.7679, -2.5183, -3.0565, -3.2350, -3.3700, -3.3302, -3.6279, -3.0913, -2.2964, -1.8721, -1.9511, -2.4879, -2.6331,
         -3.0002, -2.1385, -2.1943, -2.7402, -2.5382, -3.4318, -3.2625, -2.7020, -2.8953, -3.1516, -3.5908, -3.1563, -3.7074, -3.5619, -3.5425, -3.2412, -3.4141, -3.8322, -3.8570, -4.0342, -3.6987,
         -3.6917, -3.7528, -3.4903, -4.0536, -3.6648, -4.1839, -3.6998, -3.8443],
        [ 1.7726, -3.9808, -4.0443, -4.2974,

In [27]:
analyse_predictions(predictions2, [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

,Sentence,Best label,Best label score,Positive label
0,xr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: normal.\nheart: normal.\nright heart border: normal.\nleft heart bord...,normal,2.174707,normal
1,\nxr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: left costophrenic angle is blunted with thin stripe of homogenous op...,tuberculosis,0.686188,"tuberculosis,opacity"
2,6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:\nresults:\npost cabg status.\nno focal lesion seen in the lung parenchyma.\ncp angles and domes of the diaphragm are normal.\nboth hila are n...,normal,1.772599,normal
3,\nx-ray chest (pa view)\nthe cardio thoracic ratio is normal.\nthe heart size and configuration are within normal limits.\nthe aortic arch is normal.\nthe lung fields show normal broncho-vascular ...,normal,-0.156686,
